# Pandas and Covid-19

This notebook is an example of data analysis and manipulation with **Pandas** and has beed created in [Google Colab](https://colab.research.google.com)

Enjoy it!

In [2]:
import numpy as np
import pandas as pd

## The Data

To get some data I and going to download it from Data Repository by Johns Hopkins CSSE

https://github.com/CSSEGISandData/COVID-19

I first remove the folder where I am goint to store the data so I can re-execute this sentences without any problems ...

In [3]:
!rm -rf ./COVID-19

The dataset is avaible in GitHub so I use the `git` command to get it

In [4]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 72227 (delta 4), reused 9 (delta 4), pack-reused 72215
Receiving objects: 100% (72227/72227), 602.99 MiB | 32.08 MiB/s, done.
Resolving deltas: 100% (47504/47504), done.
Checking out files: 100% (788/788), done.


More than 600 Mb of data ...

## Exporing the data


In [7]:
!ls -lt ./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports | head

total 122836
-rw-r--r-- 1 root root 564157 Dec  7 18:29 12-06-2020.csv
-rw-r--r-- 1 root root      0 Dec  7 18:29 README.md
-rw-r--r-- 1 root root 563864 Dec  7 18:29 12-05-2020.csv
-rw-r--r-- 1 root root 563736 Dec  7 18:29 12-04-2020.csv
-rw-r--r-- 1 root root 563734 Dec  7 18:29 12-03-2020.csv
-rw-r--r-- 1 root root 563593 Dec  7 18:29 12-02-2020.csv
-rw-r--r-- 1 root root 562961 Dec  7 18:29 12-01-2020.csv
-rw-r--r-- 1 root root 562859 Dec  7 18:29 11-30-2020.csv
-rw-r--r-- 1 root root 563046 Dec  7 18:29 11-29-2020.csv


Yes!!!  
We have data files ...

Perfect. Let's explore the first dataset generated ...

In [8]:
first = pd.read_csv("./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv")

In [9]:
first.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN


And one of the last ones ...

In [10]:
last = pd.read_csv("./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/12-06-2020.csv")

In [11]:
last.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-07 05:26:14,33.93911,67.709953,47306,1874,37685,7747.0,Afghanistan,121.520719,3.961443
1,NaN,NaN,NaN,Albania,2020-12-07 05:26:14,41.15330,20.168300,42988,905,21617,20466.0,Albania,1493.779971,2.105239
2,NaN,NaN,NaN,Algeria,2020-12-07 05:26:14,28.03390,1.659600,88252,2516,57146,28590.0,Algeria,201.254050,2.850927
3,NaN,NaN,NaN,Andorra,2020-12-07 05:26:14,42.50630,1.521800,7050,78,6238,734.0,Andorra,9124.441856,1.106383
4,NaN,NaN,NaN,Angola,2020-12-07 05:26:14,-11.20270,17.873900,15591,354,8338,6899.0,Angola,47.437695,2.270541


Can I concatenate both datasets?

In [12]:
last.query("Country_Region == 'Spain'")

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
593,NaN,NaN,Andalusia,Spain,2020-12-07 05:26:14,37.544300,-4.727800,238783,4296,10671,223816.0,"Andalusia, Spain",2833.410759,1.799123
594,NaN,NaN,Aragon,Spain,2020-12-07 05:26:14,41.597600,-0.905700,74534,2360,3772,68402.0,"Aragon, Spain",5644.009553,3.166340
595,NaN,NaN,Asturias,Spain,2020-12-07 05:26:14,43.361400,-5.859300,24175,1073,1063,22039.0,"Asturias, Spain",2364.985497,4.438469
596,NaN,NaN,Baleares,Spain,2020-12-07 05:26:14,39.710358,2.995148,25288,425,1533,23330.0,"Baleares, Spain",2128.225413,1.680639
597,NaN,NaN,C. Valenciana,Spain,2020-12-07 05:26:14,39.484000,-0.753300,108586,2444,9970,96172.0,"C. Valenciana, Spain",2182.646766,2.250751
598,NaN,NaN,Canarias,Spain,2020-12-07 05:26:14,28.291600,-16.629100,21753,346,1537,19870.0,"Canarias, Spain",985.680826,1.590585
599,NaN,NaN,Cantabria,Spain,2020-12-07 05:26:14,43.182800,-3.987800,15903,324,2287,13292.0,"Cantabria, Spain",2734.160762,2.037351
600,NaN,NaN,Castilla - La Mancha,Spain,2020-12-07 05:26:14,39.279600,-3.097700,87956,3825,6392,77739.0,"Castilla - La Mancha, Spain",4322.423419,4.348765
601,NaN,NaN,Castilla y Leon,Spain,2020-12-07 05:26:14,41.835700,-4.397600,123433,4647,8716,110070.0,"Castilla y Leon, Spain",5126.523580,3.764795
602,NaN,NaN,Catalonia,Spain,2020-12-07 05:26:14,41.591200,1.520900,310141,8126,26203,275812.0,"Catalonia, Spain",4098.907398,2.620099


In [13]:
pd.concat((first, last), axis = 0)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3971,NaN,NaN,NaN,98038.0,828.0,72773.0,NaN,NaN,NaN,West Bank and Gaza,2020-12-07 05:26:14,31.952200,35.233200,24437.0,West Bank and Gaza,1921.780149,0.844570
3972,NaN,NaN,NaN,10.0,1.0,8.0,NaN,NaN,NaN,Western Sahara,2020-12-07 05:26:14,24.215500,-12.885800,1.0,Western Sahara,1.674116,10.000000
3973,NaN,NaN,NaN,2337.0,639.0,1549.0,NaN,NaN,NaN,Yemen,2020-12-07 05:26:14,15.552727,48.516388,149.0,Yemen,7.835454,27.342747
3974,NaN,NaN,NaN,17916.0,364.0,17173.0,NaN,NaN,NaN,Zambia,2020-12-07 05:26:14,-13.133897,27.849332,379.0,Zambia,97.454541,2.031704


Ups!!! The column names don't match :-(

## Loading the data into Pandas and cleaning it

In [14]:
import glob
import os

files = glob.glob("./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv")
files.sort(key=os.path.getmtime)

We are going to:
- Create a blank Dataset to store all the data
- Load every dataset unifying the column names so we can concatenate it without any problem.
- Remove extra blank spaces from the country field
- Enrich the information with the date of the data in the correct type

In [15]:
data = pd.DataFrame()
for file in files:  
  df = pd.read_csv(file).rename(columns = {'Province/State' : 'State', 
                        "Country/Region" : 'Country',
                        'Province_State' : 'State', 
                        "Country_Region" : 'Country',
                        'Last Update' : 'Last_Update',
                        'Confirmed' : 'ConfirmedAcum',
                        'Deaths' : 'DeathsAcum',
                        'Recovered' : 'RecoveredAcum'})
  df = df.assign(Date = pd.to_datetime(file[-14:-4], format = '%m-%d-%Y'),
                 Country = df.Country.str.strip())
  data = pd.concat((data, df), axis = 0)


I noticed that the country names were a little messy.   
Let's fix it ...

In [16]:
data['Country'] = data.Country.replace({'Bahamas, The' : 'Bahamas',
                         'Congo (Brazzaville)' : 'Congo',
                         'Congo (Kinshasa)' : 'Congo',
                         "Cote d'Ivoire" : "Cote d'Ivoire",
                         "Curacao" : "Curaçao",
                         'Czech Republic' : 'Czech Republic (Czechia)',
                         'Czechia' : 'Czech Republic (Czechia)',
                         'Faroe Islands' : 'Faeroe Islands',
                         'Macau' : 'Macao',
                         'Mainland China' : 'China',
                         'Palestine' : 'State of Palestine',
                         'Reunion' : 'Réunion',
                         'Saint Kitts and Nevis' : 'Saint Kitts & Nevis',
                         'Sao Tome and Principe' : 'Sao Tome & Principe',
                         'US' : 'United States',
                         'Gambia, The' : 'Gambia',
                         'Hong Kong SAR' : 'Hong Kong',
                         'Korea, South' : 'South Korea',
                         'Macao SAR' : 'Macao',
                         'Taiwan*' : 'Taiwan',
                         'Viet Nam' : 'Vietnam',
                         'West Bank and Gaza' : 'State of Palestine',
                         'occupied Palestinian territory' : 'State of Palestine'
                         })

I'm going to fill in the NULL values ​​of the 'State' and 'Admin2' fields so that later I can group the information correctly

In [18]:
data = data.fillna({'State' : 'NA', 'Admin2' : 'NA'})

Finally I am going to be left alone with the columns that interest me

In [19]:
data = data[['Date', 'Country', 'State', 'Admin2', 'ConfirmedAcum', 'DeathsAcum', 'RecoveredAcum']]

Let's verify the structure of the dateset ...

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 971477 entries, 0 to 3975
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           971477 non-null  datetime64[ns]
 1   Country        971477 non-null  object        
 2   State          971477 non-null  object        
 3   Admin2         971477 non-null  object        
 4   ConfirmedAcum  971458 non-null  float64       
 5   DeathsAcum     971035 non-null  float64       
 6   RecoveredAcum  971086 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 59.3+ MB


Wait a set, I think that can be interesting have a column the the active cases. Let's create it ...

In [21]:
data['ActiveAcum'] = data.ConfirmedAcum  - data.DeathsAcum - data.RecoveredAcum

In [22]:
data.query("Country == 'Spain'").sort_values('Date', ascending = False).head()

,Date,Country,State,Admin2,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum
612,2020-12-06,Spain,Unknown,NA,0.0,0.0,0.0,0.0
602,2020-12-06,Spain,Catalonia,NA,310141.0,8126.0,26203.0,275812.0
593,2020-12-06,Spain,Andalusia,NA,238783.0,4296.0,10671.0,223816.0
594,2020-12-06,Spain,Aragon,NA,74534.0,2360.0,3772.0,68402.0
595,2020-12-06,Spain,Asturias,NA,24175.0,1073.0,1063.0,22039.0


Perfect :-)

Now, I am going to group and summarize the data because I want to be sure that there is only one row per Date, Country, State and Admin2

In [23]:
data = data.groupby(["Date", "Country", "State", "Admin2"]).agg("sum").reset_index()

In [24]:
data.head()

,Date,Country,State,Admin2,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum
0,2020-01-22,China,Anhui,NA,1.0,0.0,0.0,0.0
1,2020-01-22,China,Beijing,NA,14.0,0.0,0.0,0.0
2,2020-01-22,China,Chongqing,NA,6.0,0.0,0.0,0.0
3,2020-01-22,China,Fujian,NA,1.0,0.0,0.0,0.0
4,2020-01-22,China,Gansu,NA,0.0,0.0,0.0,0.0


##  Daily Cases
I am going to enrich the data by creating new columns with the daily cases.  

First I create new columns with the cases from the previous day

In [25]:
data = data.sort_values(['State', 'Country', 'Date']).\
            assign(ConfirmedPrevious = data.groupby(['Admin2', 'State', 'Country']).shift(1)["ConfirmedAcum"],
                   DeathsPrevious = data.groupby(['Admin2', 'State', 'Country']).shift(1)["DeathsAcum"],
                   RecoveredPrevious = data.groupby(['Admin2', 'State', 'Country']).shift(1)["RecoveredAcum"],
                   ActivePrevious = data.groupby(['Admin2', 'State', 'Country']).shift(1)["ActiveAcum"],
            ).\
            fillna({ 'ConfirmedPrevious' : 0, 'DeathsPrevious' : 0, 'RecoveredPrevious' : 0 })

In [26]:
data.head()

,Date,Country,State,Admin2,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,ConfirmedPrevious,DeathsPrevious,RecoveredPrevious,ActivePrevious
2599,2020-02-28,Canada,"Montreal, QC",NA,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2713,2020-02-29,Canada,"Montreal, QC",NA,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2834,2020-03-01,Canada,"Montreal, QC",NA,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2961,2020-03-02,Canada,"Montreal, QC",NA,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3103,2020-03-03,Canada,"Montreal, QC",NA,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


After that I am going to assign the new fields subtracting the previous acum cases to the actual acum cases

In [27]:
data = data.assign(Confirmed = data.ConfirmedAcum -  data.ConfirmedPrevious,
            Deaths = data.DeathsAcum - data.DeathsPrevious,
            Recovered = data.RecoveredAcum - data.RecoveredPrevious,
            Active = data.ActiveAcum - data.ActivePrevious
            )

I no longer need the fields I used to make the calculation so I can drop them

In [28]:
data = data.drop(['ConfirmedPrevious', 'DeathsPrevious', 'RecoveredPrevious', 'ActivePrevious'], axis = 1)

Does the data look good?

In [29]:
data.query("Country == 'Spain'")

,Date,Country,State,Admin2,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active
172123,2020-05-14,Spain,Andalusia,NA,12359.0,1336.0,9918.0,1105.0,12359.0,1336.0,9918.0,NaN
175417,2020-05-15,Spain,Andalusia,NA,12401.0,1344.0,10148.0,909.0,42.0,8.0,230.0,-196.0
178717,2020-05-16,Spain,Andalusia,NA,12420.0,1355.0,10319.0,746.0,19.0,11.0,171.0,-163.0
182021,2020-05-17,Spain,Andalusia,NA,12420.0,1355.0,10319.0,746.0,0.0,0.0,0.0,0.0
185328,2020-05-18,Spain,Andalusia,NA,12458.0,1358.0,10671.0,429.0,38.0,3.0,352.0,-317.0
...,...,...,...,...,...,...,...,...,...,...,...,...
951941,2020-12-02,Spain,Unknown,NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
955916,2020-12-03,Spain,Unknown,NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
959891,2020-12-04,Spain,Unknown,NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
963866,2020-12-05,Spain,Unknown,NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data By Country




So far, we have data by 3 geographical levels: Country, State and a lower level called Admin2

The problem is that not all countries have this level of information, so I will create a new dataset only with the country level data

In [30]:
data_by_country = data.groupby(["Date", "Country"]).agg("sum").reset_index()
data_by_country = data_by_country.sort_values(['Country', 'Date'])
data_by_country.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52205 entries, 848 to 52204
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           52205 non-null  datetime64[ns]
 1   Country        52205 non-null  object        
 2   ConfirmedAcum  52205 non-null  float64       
 3   DeathsAcum     52205 non-null  float64       
 4   RecoveredAcum  52205 non-null  float64       
 5   ActiveAcum     52205 non-null  float64       
 6   Confirmed      52205 non-null  float64       
 7   Deaths         52205 non-null  float64       
 8   Recovered      52205 non-null  float64       
 9   Active         52205 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 4.4+ MB


In [31]:
data_by_country.head()

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active
848,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
886,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
928,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
977,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1030,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
data_by_country[data_by_country.Country == 'United States']

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active
7,2020-01-22,United States,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
22,2020-01-23,United States,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,2020-01-24,United States,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
47,2020-01-25,United States,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
62,2020-01-26,United States,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
51435,2020-12-02,United States,13921913.0,273446.0,5322128.0,8326339.0,200055.0,2804.0,95547.0,101704.0
51625,2020-12-03,United States,14139577.0,276325.0,5404018.0,8459234.0,217664.0,2879.0,81890.0,132895.0
51815,2020-12-04,United States,14367462.0,278932.0,5470389.0,8618141.0,227885.0,2607.0,66371.0,158907.0
52005,2020-12-05,United States,14581337.0,281186.0,5576026.0,8724125.0,213875.0,2254.0,105637.0,105984.0


## Cases per million inhabitants



We are going to enrich the information with the number of cases per million inhabitants, so we need population data by country.

A small internet search leads me to a page that has population data for 2020:

https://www.worldometers.info/world-population/population-by-country/

It seems that this information is protected to be downloaded automatically so I have no choice but to do it manually and upload the data to a GitHub Repository:

https://github.com/dvillaj/world-population/



I load the data to the Pandas, clean it up and just maintain the field of the population 


In [33]:
population = pd.read_excel("https://github.com/dvillaj/world-population/blob/master/data/world-popultation-2020.xlsx?raw=true", sheet_name="Data")

In [34]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            235 non-null    object 
 1   Population (2020)  235 non-null    int64  
 2   Yearly Change      235 non-null    float64
 3   Net Change         235 non-null    int64  
 4   Density (P/Km²)    235 non-null    float64
 5   Land Area (Km²)    235 non-null    int64  
 6   Migrants (net)     201 non-null    float64
 7   Fertility Rate     201 non-null    float64
 8   Average Age        201 non-null    float64
 9   Urban Pop %        222 non-null    float64
 10  World Share        235 non-null    float64
dtypes: float64(7), int64(3), object(1)
memory usage: 20.3+ KB


In [35]:
population = population.rename(columns = {
    'Population (2020)' : 'Population',
    'Yearly Change' : 'Yearly_Change',
    'Net Change' : 'Net_Change',
    'Density (P/Km²)' : 'Density',
    'Land Area (Km²)' : 'Land_Area',
    'Migrants (net)' : 'igrants',
    'Fertility Rate' : 'Fertility',
    'Average Age' : 'Mean_Age',
    'Urban Pop %' : 'Urban_Pop',
    'World Share' : 'World_Share'
})

In [36]:
population = population[['Country', 'Population']]

In [37]:
population.head()

,Country,Population
0,Afghanistan,38928346
1,Albania,2877797
2,Algeria,43851044
3,American Samoa,55191
4,Andorra,77265


Now I join the population Dataset with the country data to have the population in this dataset

In [38]:
data_by_country = data_by_country.merge(population, how = 'left', on = 'Country')

In [39]:
data_by_country.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52205 entries, 0 to 52204
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           52205 non-null  datetime64[ns]
 1   Country        52205 non-null  object        
 2   ConfirmedAcum  52205 non-null  float64       
 3   DeathsAcum     52205 non-null  float64       
 4   RecoveredAcum  52205 non-null  float64       
 5   ActiveAcum     52205 non-null  float64       
 6   Confirmed      52205 non-null  float64       
 7   Deaths         52205 non-null  float64       
 8   Recovered      52205 non-null  float64       
 9   Active         52205 non-null  float64       
 10  Population     50503 non-null  float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 4.8+ MB


In [40]:
data_by_country.head()

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active,Population
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,38928346.0
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0


And finally I calculate the number of cases per million inhabitants

In [41]:
data_by_country = data_by_country.assign(ConfirmedAcum_Millon = data_by_country.ConfirmedAcum / data_by_country.Population * 1000000)
data_by_country.head()

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active,Population,ConfirmedAcum_Millon
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,38928346.0,0.025688
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0,0.025688
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0,0.025688
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0,0.025688
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38928346.0,0.025688


## Rankins

I'm going to create a dataset of last day's cases. 

The goal is to get a set of rankings that tell me the countries with the most cases

So I need a variable that contains the last date of the dataset

In [42]:
last_day = list(data_by_country.Date.sort_values(ascending = False))[0]
last_day

Timestamp('2020-12-06 00:00:00')

Now I can filter the data by this date

In [43]:
last_day_data = data_by_country[data_by_country.Date == last_day]
last_day_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 286 to 52204
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  190 non-null    datetime64[ns]
 1   Country               190 non-null    object        
 2   ConfirmedAcum         190 non-null    float64       
 3   DeathsAcum            190 non-null    float64       
 4   RecoveredAcum         190 non-null    float64       
 5   ActiveAcum            190 non-null    float64       
 6   Confirmed             190 non-null    float64       
 7   Deaths                190 non-null    float64       
 8   Recovered             190 non-null    float64       
 9   Active                190 non-null    float64       
 10  Population            184 non-null    float64       
 11  ConfirmedAcum_Millon  184 non-null    float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 19.3+ KB


In [44]:
last_day_data

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active,Population,ConfirmedAcum_Millon
286,2020-12-06,Afghanistan,47306.0,1874.0,37685.0,7747.0,234.0,10.0,292.0,-68.0,38928346.0,1215.207037
559,2020-12-06,Albania,42988.0,905.0,21617.0,20466.0,840.0,16.0,331.0,493.0,2877797.0,14937.815280
845,2020-12-06,Algeria,88252.0,2516.0,57146.0,28590.0,750.0,15.0,529.0,206.0,43851044.0,2012.540454
1125,2020-12-06,Andorra,7050.0,78.0,6238.0,734.0,45.0,0.0,67.0,-22.0,77265.0,91244.418560
1387,2020-12-06,Angola,15591.0,354.0,8338.0,6899.0,55.0,0.0,3.0,52.0,32866272.0,474.376893
...,...,...,...,...,...,...,...,...,...,...,...,...
51191,2020-12-06,Vietnam,1366.0,35.0,1220.0,111.0,1.0,0.0,0.0,1.0,97338579.0,14.033490
51437,2020-12-06,Western Sahara,10.0,1.0,8.0,1.0,0.0,0.0,0.0,0.0,597339.0,16.740913
51678,2020-12-06,Yemen,2337.0,639.0,1549.0,149.0,33.0,6.0,2.0,25.0,29825964.0,78.354550
51942,2020-12-06,Zambia,17916.0,364.0,17173.0,379.0,18.0,0.0,5.0,13.0,18383955.0,974.545466


Let's assign new columns with the most interesting rankins ...

In [45]:
last_day_data = last_day_data.assign(
    Rank_ConfirmedAcum = last_day_data.ConfirmedAcum.rank(),
    Rank_Confirmed = last_day_data.Confirmed.rank(),
    Rank_ActiveAcum = last_day_data.ActiveAcum.rank(),
    Rank_Active = last_day_data.Active.rank(),
    Rank_ConfirmedAcum_Millon = last_day_data.ConfirmedAcum_Millon.rank()
)
last_day_data

,Date,Country,ConfirmedAcum,DeathsAcum,RecoveredAcum,ActiveAcum,Confirmed,Deaths,Recovered,Active,Population,ConfirmedAcum_Millon,Rank_ConfirmedAcum,Rank_Confirmed,Rank_ActiveAcum,Rank_Active,Rank_ConfirmedAcum_Millon
286,2020-12-06,Afghanistan,47306.0,1874.0,37685.0,7747.0,234.0,10.0,292.0,-68.0,38928346.0,1215.207037,103.0,102.0,106.0,34.0,61.0
559,2020-12-06,Albania,42988.0,905.0,21617.0,20466.0,840.0,16.0,331.0,493.0,2877797.0,14937.815280,102.0,134.0,128.0,162.0,124.0
845,2020-12-06,Algeria,88252.0,2516.0,57146.0,28590.0,750.0,15.0,529.0,206.0,43851044.0,2012.540454,117.0,131.0,142.0,146.0,73.0
1125,2020-12-06,Andorra,7050.0,78.0,6238.0,734.0,45.0,0.0,67.0,-22.0,77265.0,91244.418560,62.0,77.5,65.0,37.5,184.0
1387,2020-12-06,Angola,15591.0,354.0,8338.0,6899.0,55.0,0.0,3.0,52.0,32866272.0,474.376893,82.0,79.0,101.0,129.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51191,2020-12-06,Vietnam,1366.0,35.0,1220.0,111.0,1.0,0.0,0.0,1.0,97338579.0,14.033490,32.0,51.0,37.0,99.0,4.0
51437,2020-12-06,Western Sahara,10.0,1.0,8.0,1.0,0.0,0.0,0.0,0.0,597339.0,16.740913,4.0,25.0,7.0,74.0,5.0
51678,2020-12-06,Yemen,2337.0,639.0,1549.0,149.0,33.0,6.0,2.0,25.0,29825964.0,78.354550,40.0,71.0,42.0,118.5,17.0
51942,2020-12-06,Zambia,17916.0,364.0,17173.0,379.0,18.0,0.0,5.0,13.0,18383955.0,974.545466,86.0,65.5,55.0,110.0,51.0


Which countries have the most confirmed cases?

In [46]:
last_day_data.sort_values('Rank_ConfirmedAcum', ascending = False)[['Country', 'ConfirmedAcum']].reset_index(drop = True).head(10)

,Country,ConfirmedAcum
0,United States,14757000.0
1,India,9677203.0
2,Brazil,6603540.0
3,Russia,2439163.0
4,France,2345648.0
5,Italy,1728878.0
6,United Kingdom,1727751.0
7,Spain,1684647.0
8,Argentina,1463110.0
9,Colombia,1371103.0


Which countries have more confirmed cases on the last day?

In [47]:
last_day_data.sort_values('Rank_Confirmed', ascending = False)[['Country', 'Confirmed']].reset_index(drop = True).head(10)

,Country,Confirmed
0,United States,175663.0
1,India,32981.0
2,Turkey,30402.0
3,Russia,28701.0
4,Brazil,26363.0
5,Italy,18887.0
6,United Kingdom,17372.0
7,Ukraine,11928.0
8,Iran,11561.0
9,France,11022.0


Which countries have the most active cases?

In [48]:
last_day_data.sort_values('Rank_ActiveAcum', ascending = False)[['Country', 'ActiveAcum']].reset_index(drop = True).head(10)

,Country,ActiveAcum
0,United States,8850257.0
1,France,2115181.0
2,United Kingdom,1662673.0
3,Spain,1488019.0
4,Italy,755306.0
5,Belgium,574436.0
6,Brazil,559942.0
7,Netherlands,549560.0
8,Russia,475744.0
9,India,396729.0


Which countries had the most active cases on the last day?

In [49]:
last_day_data.sort_values('Rank_Active', ascending = False)[['Country', 'Active']].reset_index(drop = True).head(10)

,Country,Active
0,United States,126132.0
1,Turkey,26196.0
2,Brazil,24231.0
3,United Kingdom,17055.0
4,France,10620.0
5,Russia,6984.0
6,Netherlands,6853.0
7,Serbia,5751.0
8,Ukraine,5631.0
9,Hungary,5249.0


Which countries have the most active cases per million inhabitants?

In [50]:
last_day_data.sort_values('Rank_ConfirmedAcum_Millon', ascending = False)[['Country', 'ConfirmedAcum_Millon']].reset_index(drop = True).head(20)

,Country,ConfirmedAcum_Millon
0,Andorra,91244.418560
1,Montenegro,60894.874106
2,Luxembourg,59134.666075
3,San Marino,52724.647078
4,Bahrain,51675.653439
5,Czech Republic (Czechia),51063.028312
6,Belgium,51059.124184
7,Qatar,48561.411401
8,Armenia,47899.210426
9,United States,44582.724505


What is the total number of cases?

In [52]:
import locale
locale.setlocale(locale.LC_ALL, '')

total = pd.DataFrame(last_day_data[["ConfirmedAcum", "RecoveredAcum", "DeathsAcum"]].aggregate("sum")\
                     .apply(lambda value : locale.format("%d", value, grouping=True)))
total.columns = ["Total Cases"]
total.index = ['Confirmed', 'Recovered', 'Deaths']
display(total.T)


,Confirmed,Recovered,Deaths
Total Cases,"67,073,728","43,103,827","1,536,056"


## Saving the clean dataset

Finally I will create an Excel file with the information per country once it is clean and in perfect condition to apply some machine learning algorithms ...

In [54]:
data_by_country.to_excel("All_data.xlsx", index = False)

## Convert to PDF

[Google Colab](https://colab.research.google.com) does not have a 'export to PDF' feature so let's install the packages needed to export this notebook to pdf manually.

```
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc
```


Now let's mount our 'google drive' unit into this notebook so we can access to it.



``` 
from google.colab import drive
drive.mount('/content/drive')
```

Let's list the notebooks in 'Google Drive' to check that everything is fine and copy it to the local directory

```
ls -la "./drive/My Drive/Colab Notebooks/"
```

```
!cp "./drive/My Drive/Colab Notebooks/Covid-19.ipynb" .
```

The last step is generate a PDF from this notebook :-)

```
!jupyter nbconvert --to PDF "Covid-19.ipynb"
````